In [32]:
import en_core_web_md
from nltk.corpus import sentiwordnet as swn
from nltk import word_tokenize, pos_tag

In [2]:
nlp = en_core_web_md.load()

In [4]:
with open('../../../../tasks/02-structural-linguistics/data/examiner-headlines.txt', 'r') as f:
    corpus = f.readlines()

In [236]:
def has_entities(doc):
    entities = ['ORG', 'PERSON', 'GPE', 'TIME', 'MONEY', 'PRODUCT']
    ents = [x.label_ in entities for x in doc.ents]
    return any(ents)

In [220]:
def get_syns_tag(univ_tag):
    if univ_tag == 'ADJ':
        return 'a'
    if univ_tag == 'NOUN':
        return 'n'
    if univ_tag == 'VERB':
        return 'v'
    if univ_tag == 'ADV':
        return 'r'

In [182]:
# FIXME: debug case with mom abuse
def get_sentiment(doc):
    pos_sum = 0.0
    neg_sum = 0.0

    for token in doc:
#         print(token)
        pos = get_syns_tag(token.pos_)
        
        if pos:
            synsets = list(swn.senti_synsets(token.text, pos))
#             print(synsets)
            pos_score = [x.pos_score() for x in synsets]
            neg_score = [x.neg_score() for x in synsets]
            if pos_score:
                pos_agv = sum(pos_score)/len(pos_score)
                if pos_agv > 0.5:
                    pos_sum += pos_agv
            if neg_score:
                neg_agv = sum(neg_score)/len(neg_score)
                if neg_agv > 0.5:
                    neg_sum += neg_agv
    
    if pos_sum or neg_sum:
        return pos_sum > neg_sum
    return None

In [313]:
def is_adj_or_adv_comp_sup(doc):
    return any((x.pos_ == 'ADJ' or x.pos_ == 'ADV') and x.text.lower() != x.lemma_.lower() for x in doc)

In [325]:
def debug(corpus, s, e):
    all_ents = 0
    all_sents = 0
    all_super_ad = 0

    for sample in corpus[s:e]:
        doc = nlp(sample)
#         print(sample)

        has_ents = has_entities(doc)
        sents = get_sentiment(doc)
        is_super_ad = is_adj_or_adv_comp_sup(doc)
        
        if has_ents:
            all_ents += 1
        if sents:
            all_sents += 1
        if is_super_ad:
            all_super_ad += 1
    res = {
        'entities': all_ents/len(corpus[s:e]),
        'sentiment': all_sents/len(corpus[s:e]),
        'is_super_ad': all_super_ad/len(corpus[s:e])
    }
    
    return res
        
    

In [324]:
print(debug(corpus, 0, 100))

{'entities': 32.89, 'sentiment': 2.34, 'is_super_ad': 1.21}


In [17]:
list(swn.senti_synsets('happy', 'a'))

[SentiSynset('happy.a.01'),
 SentiSynset('felicitous.s.02'),
 SentiSynset('glad.s.02'),
 SentiSynset('happy.s.04')]